In [ ]:
src_name='Crab'
name1='1A 0535+262'
ra=83.633080
dec=22.014500
radius=15.
radius_jemx=5.
T1_utc='2003-03-15T23:27:40.0'
T2_utc='2018-03-16T00:03:15.0'
E1_keV=20.
E2_keV=40.
E1_keV_jemx=3.
E2_keV_jemx=20.

rebin=10

In [ ]:
import numpy as np
ebins=np.logspace(0,5,26)
print(ebins)

In [ ]:
from oda_api.api import DispatcherAPI
from oda_api.plot_tools import OdaImage,OdaLightCurve
from oda_api.data_products import BinaryData
import os
from astropy.io import fits
import numpy as np
from numpy import sqrt,exp
%matplotlib notebook
import matplotlib.pyplot as plt

host='https://www.astro.unige.ch/cdci/astrooda/dispatch-data'
disp=DispatcherAPI(host=host)

In [ ]:
image_catalog=disp.get_product(instrument='isgri',
                      product='isgri_image',
                      T1='2003-03-15T23:27:40.0',
                      T2='2015-03-16T00:03:15.0',
                      radius=radius,
                      E1_keV=E1_keV,
                      E2_keV=E2_keV,
                      osa_version='OSA10.2',
                      RA=ra,
                      DEC=dec,
                      detection_threshold=10,
                      product_type='Real')
        

In [ ]:
image_catalog.dispatcher_catalog_1.table

In [ ]:
image_catalog.dispatcher_catalog_1.table.remove_rows([2,3])

In [ ]:
image_catalog.dispatcher_catalog_1.table

In [ ]:
api_cat=image_catalog.dispatcher_catalog_1.get_api_dictionary()

In [ ]:
lc=disp.get_product(instrument='isgri',
                    product='isgri_lc',
                    T1=T1_utc,
                    T2=T2_utc,
                    radius=radius,
                    query_type='Real',
                    osa_version='OSA10.2',
                    RA=ra,
                    DEC=dec,
                    product_type='Real',
                    selected_catalog=api_cat)



In [ ]:
t=[]
r=[]
err=[]
tot_counts=[]
backv=[]
backe=[]
i=0
for ID,s in enumerate(lc._p_list):
    if s.meta_data['src_name']==src_name:
        i=i+1
        for tt in s.data_unit[1].data['TIME']:
            t.append(tt)
        for rr in s.data_unit[1].data['RATE']:
            r.append(rr)
        for ee in s.data_unit[1].data['ERROR']:
            err.append(ee)
        for tc in s.data_unit[1].data['TOT_COUNTS']:
            tot_counts.append(tc)
        for bv in s.data_unit[1].data['BACKV']:
            backv.append(bv)
        for be in s.data_unit[1].data['BACKE']:
            backe.append(be)
t=np.array(t)
r=np.array(r)
err=np.array(err)
tot_counts=np.array(tot_counts)
backv=np.array(backv)
backe=np.array(backe)

t1=[]
r1=[]
err1=[]
tot_counts1=[]
backv1=[]
backe1=[]
i=0
for ID,s in enumerate(lc._p_list):
    if s.meta_data['src_name']==name1:
        i=i+1
        for tt in s.data_unit[1].data['TIME']:
            t1.append(tt)
        for rr in s.data_unit[1].data['RATE']:
            r1.append(rr)
        for ee in s.data_unit[1].data['ERROR']:
            err1.append(ee)
        for tc in s.data_unit[1].data['TOT_COUNTS']:
            tot_counts1.append(tc)
        for bv in s.data_unit[1].data['BACKV']:
            backv1.append(bv)
        for be in s.data_unit[1].data['BACKE']:
            backe1.append(be)
t1=np.array(t1)
r1=np.array(r1)
err1=np.array(err1)
tot_counts1=np.array(tot_counts1)
backv1=np.array(backv1)
backe1=np.array(backe1)



In [ ]:
fig = plt.figure(figsize=(10,7))
t0=51544
plt.errorbar(t+t0,r,yerr=err,linestyle='none',color='black',alpha=1,linewidth=2, label='Crab')
plt.errorbar(t1+t0,r1,yerr=err1,linestyle='none',color='blue',alpha=1,linewidth=2, label=name1)

plt.tick_params(axis='both', which='major', labelsize=16)
ymax=1.2*max(r+err)
ymin=0.5*max(r-err)

tmin=t[0]+t0
tmax=t[-1]+t0
#plt.ylim(ymin,ymax)
#plt.xlim(55297,55298)
plt.xlabel('Time, MJD',fontsize=16)
plt.ylabel('Rate, cts/s',fontsize=16)
#plt.text(tmin,0.95*ymax, str(E1_keV)+'-'+str(E2_keV)+' keV',fontsize=16)
#plt.text(tmin,0.9*ymax, src_name,fontsize=16)

name=src_name.replace(" ", "")
plt.legend()
plt.savefig(name+'_lc_50scws.pdf',format='pdf',dpi=100)

In [ ]:
image_catalog_jemx=disp.get_product(instrument='jemx',
                      jemx_num=1,
                      product='jemx_image',
                      T1=T1_utc,
                      T2=T2_utc,
                      radius=radius_jemx,
                      E1_keV=E1_keV_jemx,
                      E2_keV=E2_keV_jemx,
                      osa_version='OSA10.2',
                      RA=ra,
                      DEC=dec,
                      detection_threshold=10,
                      product_type='Real')

In [ ]:
image_catalog_jemx.dispatcher_catalog_1.table

In [ ]:
#image_catalog_jemx.dispatcher_catalog_1.table.remove_rows([2,3,4])

In [ ]:
image_catalog_jemx.dispatcher_catalog_1.table

In [ ]:
api_cat_jemx=image_catalog_jemx.dispatcher_catalog_1.get_api_dictionary()

In [ ]:
spectrum_results_jemx=[]

for i in range(2004,2020):
    T1_utc=str(i)+'-01-01T00:00:00.0'
    T2_utc=str(i)+'-12-31T23:59:59.0'
        
    print(T1_utc,'-',T2_utc)
    if i<2016:
        osa_version='OSA10.2'
    else:
        osa_version='OSA11.0'
    data=disp.get_product(instrument='jemx',
                        jemx_num=1,
                        product='jemx_spectrum',
                        T1=T1_utc,
                        T2=T2_utc,
                        radius=radius_jemx,
                        query_type='Real',
                        osa_version='OSA10.2',
                        RA=ra,
                        DEC=dec,
                        product_type='Real',
                        selected_catalog=api_cat_jemx)
    
    spectrum_results_jemx.append(data)



In [ ]:
i=2003
for data in spectrum_results_jemx:
    print(i)
    for ID,s in enumerate(data._p_list):
        if (s.meta_data['src_name']==src_name):
            if(s.meta_data['product']=='jemx_spectrum'):
                ID_spec_jemx1=ID
            if(s.meta_data['product']=='jemx_arf'):
                ID_arf_jemx1=ID
            if(s.meta_data['product']=='jemx_rmf'):
                ID_rmf_jemx1=ID

    print(i+2003, ID_spec_jemx1, ID_arf_jemx1, ID_rmf_jemx1)
    name=src_name+'_'+str(i)

    specname_jemx1=name+'_spectrum_osa10_jemx1.fits'
    arfname_jemx1=name+'_arf_osa10_jemx1.fits.gz'
    rmfname_jemx1=name+'_rmf_osa10_jemx1.fits.gz'
    data._p_list[ID_spec_jemx1].write_fits_file(specname_jemx1)
    data._p_list[ID_arf_jemx1].write_fits_file(arfname_jemx1)
    data._p_list[ID_rmf_jemx1].write_fits_file(rmfname_jemx1)
    
    tmp_f = fits.open(name+'_spectrum_osa10_jemx1.fits', mode='update')
    
    tmp_f[1].data['SYS_ERR'] += 0.05
    
    tmp_f.flush()
    tmp_f.close()
    
    i+=1



In [ ]:
spectrum_results=[]

for i in range(2004,2020):
    T1_utc=str(i)+'-01-01T00:00:00.0'
    T2_utc=str(i)+'-12-31T23:59:59.0'
    
    print(T1_utc,'-',T2_utc)
    if i<2016:
        osa_version='OSA10.2'
    else:
        osa_version='OSA11.0'
    data=disp.get_product(instrument='isgri',
                        product='isgri_spectrum',
                        T1=T1_utc,
                        T2=T2_utc,
                        radius=radius,
                        query_type='Real',
                        osa_version=osa_version,
                        RA=ra,
                        DEC=dec,
                        product_type='Real',
                        selected_catalog=api_cat)
    for ID,s in enumerate(data._p_list):
        if (s.meta_data['src_name']==src_name):
            if(s.meta_data['product']=='isgri_spectrum'):
                ID_spec=ID
            if(s.meta_data['product']=='isgri_arf'):
                ID_arf=ID
            if(s.meta_data['product']=='isgri_rmf'):
                ID_rmf=ID

    print(ID_spec, ID_arf, ID_rmf)
    spec=data._p_list[ID_spec].data_unit[1].data
    arf=data._p_list[ID_arf].data_unit[1].data
    rmf=data._p_list[ID_rmf].data_unit[2].data
    expos=data._p_list[0].data_unit[1].header['EXPOSURE']
    name=src_name+'_'+str(i)
    specname=name+'_spectrum_osa10.fits'
    arfname=name+'_arf_osa10.fits.gz'
    rmfname=name+'_rmf_osa10.fits.gz'
    data._p_list[ID_spec].write_fits_file(specname)
    data._p_list[ID_arf].write_fits_file(arfname)
    data._p_list[ID_rmf].write_fits_file(rmfname)
    hdul = fits.open(specname, mode='update')
    hdr=hdul[1].header
    hdr.set('EXPOSURE', expos)
    hdul.close()
    !./spectrum_fit_osa10_isgri_jemx1.sh $name $rebin
    
    spectrum_results.append(data)


In [ ]:
plt.figure(figsize=(10,7))

for i in range(2004,2019):
    name=src_name+'_'+str(i)
    spectrum=np.genfromtxt(name+'_spectrum_osa10.txt',skip_header=3)
    en=spectrum[:,0]
    en_err=spectrum[:,1]
    fl=spectrum[:,2]
    fl_err=spectrum[:,3]
    mo=spectrum[:,4]
    plt.errorbar(en,fl,xerr=en_err,yerr=fl_err,linestyle='none',linewidth=8,color='blue',alpha=0.1)
    plt.plot(en,mo,color='black',linewidth=4,alpha=0.1)


plt.tick_params(axis='both', which='major', labelsize=16)
plt.text(5,10,'JEMX1',fontsize=16)
plt.text(50,8,'ISGRI',fontsize=16)

plt.xscale('log')
plt.yscale('log')
plt.ylim(1,20.)
plt.xlim(2,350)
plt.xlabel('$E$, keV',fontsize=16)
plt.ylabel('$E^2F_E$, keV/(cm$^2$s)',fontsize=16)
plt.legend(loc='lower left',fontsize=16)
plt.savefig(src_name+'_spectrum1.pdf',format='pdf',dpi=100)


In [ ]:
T1_utc='2010-04-01T00:00:00'
T2_utc='2010-04-21T00:00:00'
lc=disp.get_product(instrument='isgri',
                    product='isgri_lc',
                    T1=T1_utc,
                    T2=T2_utc,
                    query_type='Real',
                    osa_version='OSA10.2',
                    RA=ra,
                    DEC=dec,
                    product_type='Real',
                    selected_catalog=api_cat)



In [ ]:
t=[]
r=[]
err=[]
tot_counts=[]
backv=[]
backe=[]
i=0
for ID,s in enumerate(lc._p_list):
    if s.meta_data['src_name']==src_name:
        i=i+1
        for tt in s.data_unit[1].data['TIME']:
            t.append(tt)
        for rr in s.data_unit[1].data['RATE']:
            r.append(rr)
        for ee in s.data_unit[1].data['ERROR']:
            err.append(ee)
        for tc in s.data_unit[1].data['TOT_COUNTS']:
            tot_counts.append(tc)
        for bv in s.data_unit[1].data['BACKV']:
            backv.append(bv)
        for be in s.data_unit[1].data['BACKE']:
            backe.append(be)
t=np.array(t)
r=np.array(r)
err=np.array(err)
tot_counts=np.array(tot_counts)
backv=np.array(backv)
backe=np.array(backe)

t1=[]
r1=[]
err1=[]
tot_counts1=[]
backv1=[]
backe1=[]
i=0
for ID,s in enumerate(lc._p_list):
    if s.meta_data['src_name']==name1:
        i=i+1
        for tt in s.data_unit[1].data['TIME']:
            t1.append(tt)
        for rr in s.data_unit[1].data['RATE']:
            r1.append(rr)
        for ee in s.data_unit[1].data['ERROR']:
            err1.append(ee)
        for tc in s.data_unit[1].data['TOT_COUNTS']:
            tot_counts1.append(tc)
        for bv in s.data_unit[1].data['BACKV']:
            backv1.append(bv)
        for be in s.data_unit[1].data['BACKE']:
            backe1.append(be)
t1=np.array(t1)
r1=np.array(r1)
err1=np.array(err1)
tot_counts1=np.array(tot_counts1)
backv1=np.array(backv1)
backe1=np.array(backe1)



In [ ]:
#special fix to a period when 1A 0535+262 was flaring:
fig = plt.figure(figsize=(10,7))
t0=51544
plt.errorbar(t+t0,r,yerr=err,linestyle='dashed',color='black',alpha=1,linewidth=2)
plt.errorbar(t1+t0,r1,yerr=err1,linestyle='solid',color='blue',alpha=1,linewidth=2)

plt.tick_params(axis='both', which='major', labelsize=16)
ymax=1.2*max(r+err)
ymin=0.5*max(r-err)

tmin=t[0]+t0
tmax=t[-1]+t0
#plt.ylim(ymin,ymax)
plt.xlim(55289,55305)
plt.xlabel('Time, MJD',fontsize=16)
plt.ylabel('Rate, cts/s',fontsize=16)
#plt.text(tmin,0.95*ymax, str(E1_keV)+'-'+str(E2_keV)+' keV',fontsize=16)
#plt.text(tmin,0.9*ymax, src_name,fontsize=16)

name=src_name.replace(" ", "")
plt.savefig(name+'_lc_50scws.pdf',format='pdf',dpi=100)

In [ ]:
spectrum_1A0535_burst=disp.get_product(instrument='isgri',
                        product='isgri_spectrum',
                        T1='2010-04-03T00:00:00',
                        T2='2010-04-06T00:00:00',
                        query_type='Real',
                        osa_version=osa_version,
                        RA=ra,
                        DEC=dec,
                        product_type='Real',
                        selected_catalog=api_cat)

In [ ]:
for ID,s in enumerate(spectrum_1A0535_burst._p_list):
    if (s.meta_data['src_name']==src_name):
        if(s.meta_data['product']=='isgri_spectrum'):
            ID_spec=ID
        if(s.meta_data['product']=='isgri_arf'):
            ID_arf=ID
        if(s.meta_data['product']=='isgri_rmf'):
            ID_rmf=ID

print(ID_spec, ID_arf, ID_rmf)
spec=data._p_list[ID_spec].data_unit[1].data
arf=data._p_list[ID_arf].data_unit[1].data
rmf=data._p_list[ID_rmf].data_unit[2].data
expos=data._p_list[0].data_unit[1].header['EXPOSURE']
name=src_name+'_with1A0535'
specname=name+'_spectrum_osa10.fits'
arfname=name+'_arf_osa10.fits.gz'
rmfname=name+'_rmf_osa10.fits.gz'
data._p_list[ID_spec].write_fits_file(specname)
data._p_list[ID_arf].write_fits_file(arfname)
data._p_list[ID_rmf].write_fits_file(rmfname)
hdul = fits.open(specname, mode='update')
hdr=hdul[1].header
hdr.set('EXPOSURE', expos)
hdul.close()
!./spectrum_fit_osa10.sh $name $rebin



In [ ]:
for ID,s in enumerate(spectrum_1A0535_burst._p_list):
    if (s.meta_data['src_name']==name1):
        if(s.meta_data['product']=='isgri_spectrum'):
            ID_spec=ID
        if(s.meta_data['product']=='isgri_arf'):
            ID_arf=ID
        if(s.meta_data['product']=='isgri_rmf'):
            ID_rmf=ID

print(ID_spec, ID_arf, ID_rmf)
spec=data._p_list[ID_spec].data_unit[1].data
arf=data._p_list[ID_arf].data_unit[1].data
rmf=data._p_list[ID_rmf].data_unit[2].data
expos=data._p_list[0].data_unit[1].header['EXPOSURE']
name=name1+'_with1A0535'
specname=name+'_spectrum_osa10.fits'
arfname=name+'_arf_osa10.fits.gz'
rmfname=name+'_rmf_osa10.fits.gz'
data._p_list[ID_spec].write_fits_file(specname)
data._p_list[ID_arf].write_fits_file(arfname)
data._p_list[ID_rmf].write_fits_file(rmfname)
hdul = fits.open(specname, mode='update')
hdr=hdul[1].header
hdr.set('EXPOSURE', expos)
hdul.close()
!./spectrum_fit_osa10.sh $name $rebin



In [ ]:
image_catalog.dispatcher_catalog_1.table

In [ ]:
image_catalog.dispatcher_catalog_1.table.remove_rows([0])


In [ ]:
image_catalog.dispatcher_catalog_1.table

In [ ]:
api_cat=image_catalog.dispatcher_catalog_1.get_api_dictionary()

In [ ]:
spectrum_1A0535_burst1=disp.get_product(instrument='isgri',
                        product='isgri_spectrum',
                        T1='2010-04-03T00:00:00',
                        T2='2010-04-06T00:00:00',
                        query_type='Real',
                        osa_version=osa_version,
                        RA=ra,
                        DEC=dec,
                        product_type='Real',
                        selected_catalog=api_cat)

In [ ]:
for ID,s in enumerate(spectrum_1A0535_burst1._p_list):
    if (s.meta_data['src_name']==src_name):
        if(s.meta_data['product']=='isgri_spectrum'):
            ID_spec=ID
        if(s.meta_data['product']=='isgri_arf'):
            ID_arf=ID
        if(s.meta_data['product']=='isgri_rmf'):
            ID_rmf=ID

print(ID_spec, ID_arf, ID_rmf)
spec=data._p_list[ID_spec].data_unit[1].data
arf=data._p_list[ID_arf].data_unit[1].data
rmf=data._p_list[ID_rmf].data_unit[2].data
expos=data._p_list[0].data_unit[1].header['EXPOSURE']
name=src_name+'_without1A0535'
specname=name+'_spectrum_osa10.fits'
arfname=name+'_arf_osa10.fits.gz'
rmfname=name+'_rmf_osa10.fits.gz'
data._p_list[ID_spec].write_fits_file(specname)
data._p_list[ID_arf].write_fits_file(arfname)
data._p_list[ID_rmf].write_fits_file(rmfname)
hdul = fits.open(specname, mode='update')
hdr=hdul[1].header
hdr.set('EXPOSURE', expos)
hdul.close()
!./spectrum_fit_osa10.sh $name $rebin




In [ ]:
plt.figure(figsize=(10,7))

name=src_name+'_with1A0535'
spectrum=np.genfromtxt(name+'_spectrum_osa10.txt',skip_header=3)
en=spectrum[:,0]
en_err=spectrum[:,1]
fl=spectrum[:,2]
fl_err=spectrum[:,3]
mo=spectrum[:,4]
plt.errorbar(en,fl,xerr=en_err,yerr=fl_err,linestyle='none',linewidth=8,color='blue',alpha=0.1)
plt.plot(en,mo,color='black',linewidth=4,alpha=0.1)
name=src_name+'_without1A0535'
spectrum=np.genfromtxt(name+'_spectrum_osa10.txt',skip_header=3)
en=spectrum[:,0]
en_err=spectrum[:,1]
fl=spectrum[:,2]
fl_err=spectrum[:,3]
mo=spectrum[:,4]
plt.errorbar(en,fl,xerr=en_err,yerr=fl_err,linestyle='none',linewidth=8,color='red',alpha=0.1)
plt.plot(en,mo,color='black',linewidth=4,alpha=0.1)



plt.tick_params(axis='both', which='major', labelsize=16)
plt.xscale('log')
plt.yscale('log')
plt.ylim(2,20.)
plt.xlim(15,350)
plt.xlabel('$E$, keV',fontsize=16)
plt.ylabel('$E^2F_E$, keV/(cm$^2$s)',fontsize=16)
plt.legend(loc='lower left',fontsize=16)
plt.savefig(src_name+'_spectrum.pdf',format='pdf',dpi=100)

